<a href="https://colab.research.google.com/github/seph0916/seph-solo-machine/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A05%EB%8B%A8%EC%9B%90_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 트리의 앙상블
# 앙상블 학습-> 정형데이터를 다루기에 가장 뛰어난 성과를 내는 알고리즘이다.
# 비정형 데이터를 다루기에는 신경망 알고리즘이 괜찮음.

In [17]:
# 앙상블 학습의 대표주자인 랜덤 포레스트
# 랜덤포레스트 -> 트리를 훈련하기위해 데이터를 추출하게되는데 중복추출이 될수도 있다 ,기본적으로 100개의 트리를 훈련
# 이렇게 만들어진 샘플을 부트스트랩 샘플 이라고 부름 , 기본적으로 부트스트랩 샘플과 훈련세트의 크기는 같게 함
# 또한 랜덤포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 떄문에 훈련세트 과적합되는걸 막아주며 안정적 성능을 얻을수 있다.

In [7]:
# input 데이터와 target 데이터를 나누기 위해 구분
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine=pd.read_csv('https://bit.ly/wine-date')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine[['class']].to_numpy()
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=42)

In [9]:
# return_train_score =True -> 검증점수 뿐 아니라 훈련점수도 반환하게 해주는 매개변수
scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [12]:
# 결정 트리의 큰 장점 -> 특성 중요도를 계산함, 특성중요도는 각 결정 트리의 특성 중요도를 취합한것.
# 각각 알코올 , 당도 ,pH 순 -> 1절 결정 트리에서 만든 특성 중요도와 다름
# 당도의 중요도가 상당히 높았으나 지금은 낮아진상태-> 이는 하나의 특성에대해 과도하게 적합되려는것을 방지하기 위해 특성일부를 랜덤하게 선택하여 훈련
rf.fit(train_input,train_target)
print(rf.feature_importances_)

<ipython-input-12-d16175f611b7>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


[0.23167441 0.50039841 0.26792718]


In [13]:
# 랜덤 포레스트분류는 자체적으로 모델을 평가하는 점수를 얻을수 있다.
# 부트 스트랩 샘플에 포함되지 않고 남는 샘플이 있는데 이를 OOB(out of bag)샘플이라고 함 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 트리를 평가할수있음
# 마치 검증세트와 같은 역할
# 매개변수 oob_score=True 설정하면 됨

In [14]:
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)

<ipython-input-14-e256c5636b53>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train_input,train_target)


RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

In [16]:
print(rf.oob_score_)

0.8934000384837406


In [20]:
# 엑스트라 트리 랜덤 포레스트와 비슷함, 차이점: 부트스트랩 샘플을 사용하지 않음
# 즉 전체 훈련세트를 사용 DecisionTreeClassifier에서 splitter 매개변수를 random으로 지정. 하는 결정트리임.
# 가장 좋은 분할을 찾는것이 아니라 무작위로 분할함-> 무작위 분할은 성능이 낮아지겠지만 과대적합을 막고 검증세트의 점수를 높이는 효과가 있다.

In [24]:
# extra모델의 교차 검증점수 확인
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [25]:
scores

{'fit_time': array([0.84943771, 0.9349227 , 0.64622951, 0.7104342 , 0.63008213]),
 'score_time': array([0.11862397, 0.12741375, 0.09537435, 0.08810282, 0.05628443]),
 'test_score': array([0.88365385, 0.87884615, 0.90375361, 0.88835419, 0.88931665]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}

In [26]:
et.fit(train_input,train_target)
print(et.feature_importances_)

<ipython-input-26-d33477e215d5>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  et.fit(train_input,train_target)


[0.20183568 0.52242907 0.27573525]


In [29]:
# 그레디언트 부스팅
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식, 기본적으로 깊이3, 결정트리100개 사용
# 얕은 결정트리를 사용하기 떄문에 과대적합에 강함, 경사하강법 사용

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015
